In [1]:
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams.update({"font.size": 12})
%matplotlib inline

In [ ]:
!pip3 install flair

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv("/content/drive/MyDrive/WSB/WSB_top_users_features.csv", sep=";")
df.tail(1)

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity
7252,zxc123zxc123,12,Is WSB hiding the scale of financial crisis? +...,There’s a bubble they’d like us to think WE ca...,2021-10-13 2021-06-04 2021-06-04 2021-06-04 20...,13 4 4 4 29 22 19 19 7 29 23 23,10 6 6 6 4 1 1 1 1 12 12 12,2021 2021 2021 2021 2021 2021 2021 2021 2021 2...,NaN,{},Meme Gain Meme Gain News Meme Meme Meme Discus...,9.0,108,2.083333,25,0.980833,11.77,"['Meme', 'Gain', 'Meme', 'Gain', 'News', 'Meme...","{'Meme': 6, 'Gain': 2, 'News': 1, 'Discussion'...",0.515,0.59,-0.227381,0.561012,"['Is', 'WSB', 'hiding', 'the', 'scale', 'of', ...","['There', '’s', 'a', 'bubble', 'they', '’d', '...",0.287619


# Embedding of titles by user

In [7]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings
from flair.data import Sentence
from sklearn.manifold import TSNE
%matplotlib inline


## Visualize Embeddings for clustering

## Stacked Embeddings

In [9]:
from flair.embeddings import  FlairEmbeddings
# initialize the word embeddings
glove_embedding = WordEmbeddings('glove')

# initialize the document embeddings, mode = mean
document_embeddings = DocumentPoolEmbeddings([glove_embedding])
# init Flair forward and backwards embeddings
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')

2021-12-26 17:36:36,015 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmplmlea279


100%|██████████| 160000128/160000128 [00:06<00:00, 25998870.03B/s]

2021-12-26 17:36:42,480 copying /tmp/tmplmlea279 to cache at /root/.flair/embeddings/glove.gensim.vectors.npy


2021-12-26 17:36:42,804 removing temp file /tmp/tmplmlea279
2021-12-26 17:36:43,323 https://flair.informatik.hu-berlin.de/resources/embeddings/token/glove.gensim not found in cache, downloading to /tmp/tmpegklw76a


100%|██████████| 21494764/21494764 [00:01<00:00, 14792396.61B/s]

2021-12-26 17:36:45,096 copying /tmp/tmpegklw76a to cache at /root/.flair/embeddings/glove.gensim


2021-12-26 17:36:45,132 removing temp file /tmp/tmpegklw76a


In [10]:
from flair.embeddings import StackedEmbeddings

# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
stacked_embeddings = StackedEmbeddings([
                                        glove_embedding,
                                        flair_embedding_forward,
                                        flair_embedding_backward,
                                       ])

In [11]:
def get_stackedsentence_emb(text):
  sentence = Sentence(text)
  stacked_embeddings.embed(sentence)
  return sentence.embedding


# now check out the embedded tokens.
#for token in sentence:
#    print(token)
#    print(token.embedding)

In [ ]:
#df['title_emb_stacked'] = df['title'].apply(lambda x: get_stackedsentence_emb(x))
title_emb_stacked = df.loc[:50,'title'].apply(lambda x: get_stackedsentence_emb(x))

In [ ]:
from sklearn.manifold import TSNE
%matplotlib inline
def plot_embeddings(df_name,label_name,emb_col_name,total_points):

	all_users = []
	all_users_emb = []
	for index, row in df_name.iterrows():
		 #print(row["author"], row["title_emb"])
		 all_users.append(row[label_name])
		 all_users_emb.append(row[emb_col_name].numpy())
		 

	tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
	new_values = tsne_model.fit_transform(all_users_emb)

	x = []
	y = []
	for value in new_values:
		x.append(value[0])
		y.append(value[1])
			
	plt.figure(figsize=(16, 16)) 
	for i in range(len(x[0:total_points])):
		plt.scatter(x[i],y[i])
		plt.annotate(all_users[i],
						 xy=(x[i], y[i]),
						 xytext=(5, 2),
						 textcoords='offset points',
						 ha='right',
						 va='bottom')
	plt.show()

In [ ]:
#plot_embeddings(df,"author","title_emb_stacked",100)

In [ ]:
#plot_embeddings(df[df['score'] > 250],"author","title_emb_stacked",50)

## Count Years, Months 

In [13]:
from collections import Counter

In [14]:
df['year'] = df['year'].apply(lambda x: list(x.split(" ")))
df["years_count"] = df['year'].apply(lambda x: dict(Counter(list(x))))

In [15]:
df['month'] = df['month'].apply(lambda x: list(x.split(" ")))
df["months_count"] = df['month'].apply(lambda x: dict(Counter(list(x))))

In [17]:
df['date'] = df['date'].apply(lambda x: list(x.split(" ")))
df["dates_count"] = df['date'].apply(lambda x: dict(Counter(list(x))))

In [18]:
df.head(1)

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
0,-A-Brocoma2021,14,"Strength of a silverback gorilla, dominance of...",What if everyone did? Bought just one more s...,"[2021-04-03, 2021-03-28, 2021-03-16, 2021-03-1...",3 28 16 16 10 10 10 27 26 25 25 23 20 2,"[4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2]","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...",['GME'],{'GME': 1},Meme Discussion Meme YOLO Technical Analysis G...,5.428571,76,19.285714,270,0.972857,13.62,"['Meme', 'Discussion', 'Meme', 'YOLO', 'Techni...","{'Meme': 2, 'Discussion': 2, 'YOLO': 8, 'Techn...",0.22166,0.507253,0.25,0.25,"['Strength', 'of', 'a', 'silverback', 'gorilla...","[' ', 'What', 'if', 'everyone', 'did', '?', '...",0.47166,{'2021': 14},"{'4': 1, '3': 6, '2': 7}","{'2021-04-03': 1, '2021-03-28': 1, '2021-03-16..."


In [19]:
df.to_csv("/content/drive/MyDrive/WSB/WSB_top_users_features.csv",sep=';')

## Are there differences between "old" users (pre-2021) and new ones?

In [53]:
# split dataset: pre and post hype 
#users_pre = df['year'].apply(lambda x:list(set(x).intersection(pre_years)))
users_post = df[['2021' in x for x in df['year'].values] and ['2020' not in x for x in df['year'].values] and ['2018' not in x for x in df['year'].values] and ['2019' not in x for x in df['year'].values]]
users_post.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5919 entries, 0 to 7252
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              5919 non-null   object 
 1   id                  5919 non-null   int64  
 2   title               5919 non-null   object 
 3   selftext            5919 non-null   object 
 4   date                5919 non-null   object 
 5   day                 5919 non-null   object 
 6   month               5919 non-null   object 
 7   year                5919 non-null   object 
 8   tickers_            2497 non-null   object 
 9   tickers_count_      5919 non-null   object 
 10  link_flair_text     5919 non-null   object 
 11  comments_mean       5919 non-null   float64
 12  comments_sum        5919 non-null   int64  
 13  score_mean          5919 non-null   float64
 14  score_sum           5919 non-null   int64  
 15  upvote_ratio_mean   5919 non-null   float64
 16  upvote

In [58]:
users_post.to_csv("/content/drive/MyDrive/WSB/WSB_users_post-hype.csv",sep=';')

In [54]:
users_pre = df[['2021' not in x for x in df['year'].values]]
users_pre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1034 entries, 2 to 7250
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              1034 non-null   object 
 1   id                  1034 non-null   int64  
 2   title               1034 non-null   object 
 3   selftext            1034 non-null   object 
 4   date                1034 non-null   object 
 5   day                 1034 non-null   object 
 6   month               1034 non-null   object 
 7   year                1034 non-null   object 
 8   tickers_            448 non-null    object 
 9   tickers_count_      1034 non-null   object 
 10  link_flair_text     1034 non-null   object 
 11  comments_mean       1034 non-null   float64
 12  comments_sum        1034 non-null   int64  
 13  score_mean          1034 non-null   float64
 14  score_sum           1034 non-null   int64  
 15  upvote_ratio_mean   1034 non-null   float64
 16  upvote

In [55]:
users_pre.head(1)

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
2,-F1sh-,11,Very Bullish jobs chart. SPY 400 on the horizo...,https://twitter.com/espn_esports/s...,"[2020-12-04, 2020-09-08, 2020-09-07, 2020-09-0...",4 8 7 7 27 15 28 24 8 7 31,"[12, 9, 9, 9, 6, 6, 4, 4, 4, 4, 3]","[2020, 2020, 2020, 2020, 2020, 2020, 2020, 202...",NaN,{},Discussion Shitpost Meme Meme YOLO Meme Discus...,6.090909,67,1.0,11,0.545455,6.0,"['Discussion', 'Shitpost', 'Meme', 'Meme', 'YO...","{'Discussion': 2, 'Shitpost': 5, 'Meme': 3, 'Y...",0.022222,0.111111,0.074583,0.450407,"['Very', 'Bullish', 'jobs', 'chart', '.', 'SPY...","[' ', 'https://twitter.com/espn_esp...",0.096805,{'2020': 11},"{'12': 1, '9': 3, '6': 2, '4': 4, '3': 1}","{'2020-12-04': 1, '2020-09-08': 1, '2020-09-07..."


In [59]:
users_pre.to_csv("/content/drive/MyDrive/WSB/WSB_users_pre-hype.csv",sep=';')

In [56]:
users_pre_post = df[['2021' in x for x in df['year'].values] and ['2020' in x for x in df['year'].values] and ['2018' in x for x in df['year'].values] and ['2019' in x for x in df['year'].values]]
users_pre_post.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334 entries, 4 to 7250
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              1334 non-null   object 
 1   id                  1334 non-null   int64  
 2   title               1334 non-null   object 
 3   selftext            1334 non-null   object 
 4   date                1334 non-null   object 
 5   day                 1334 non-null   object 
 6   month               1334 non-null   object 
 7   year                1334 non-null   object 
 8   tickers_            693 non-null    object 
 9   tickers_count_      1334 non-null   object 
 10  link_flair_text     1334 non-null   object 
 11  comments_mean       1334 non-null   float64
 12  comments_sum        1334 non-null   int64  
 13  score_mean          1334 non-null   float64
 14  score_sum           1334 non-null   int64  
 15  upvote_ratio_mean   1334 non-null   float64
 16  upvote

In [57]:
users_pre_post.head(1)

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
4,-Hyre,19,The age of silver. And clandestine inflation. ...,You're all front row to the Fed money printing...,"[2021-01-29, 2021-01-29, 2021-01-28, 2019-12-0...",29 29 28 4 2 14 24 18 17 9 8 7 3 3 3 2 27 23 20,"[1, 1, 1, 12, 12, 11, 10, 10, 10, 10, 10, 10, ...","[2021, 2021, 2021, 2019, 2019, 2019, 2019, 201...","['AMZN', 'MU', 'ROKU', 'GME', 'BYND']","{'AMZN': 1, 'MU': 1, 'ROKU': 1, 'GME': 1, 'BYN...",DD Discussion DD YOLO Options YOLO Discussion ...,333.842105,6343,164.315789,3122,0.156316,2.97,"['DD', 'Discussion', 'DD', 'YOLO', 'Options', ...","{'DD': 3, 'Discussion': 2, 'YOLO': 5, 'Options...",-0.15,0.3,0.036943,0.479711,"['The', 'age', 'of', 'silver', '.', 'And', 'cl...","['You', ""'re"", 'all', 'front', 'row', 'to', 't...",-0.113057,"{'2021': 3, '2019': 16}","{'1': 3, '12': 2, '11': 1, '10': 10, '9': 3}","{'2021-01-29': 2, '2021-01-28': 1, '2019-12-04..."


In [60]:
users_pre_post.to_csv("/content/drive/MyDrive/WSB/WSB_users_pre-post_hype.csv",sep=';')

## Can we identify those that have posted (successful) DDs?

In [61]:
## get all users with 'DD' flair and high upvotes, positive sentiment 
users_DD = df[['DD' not in x for x in df['flairs'].values]]
users_DD.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3767 entries, 0 to 7252
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              3767 non-null   object 
 1   id                  3767 non-null   int64  
 2   title               3767 non-null   object 
 3   selftext            3767 non-null   object 
 4   date                3767 non-null   object 
 5   day                 3767 non-null   object 
 6   month               3767 non-null   object 
 7   year                3767 non-null   object 
 8   tickers_            1377 non-null   object 
 9   tickers_count_      3767 non-null   object 
 10  link_flair_text     3767 non-null   object 
 11  comments_mean       3767 non-null   float64
 12  comments_sum        3767 non-null   int64  
 13  score_mean          3767 non-null   float64
 14  score_sum           3767 non-null   int64  
 15  upvote_ratio_mean   3767 non-null   float64
 16  upvote

In [62]:
users_DD.describe()

,id,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,title_polarity,title_subjectivity,body_polarity,body_subjectivity,polarity
count,3767.000000,3767.000000,3.767000e+03,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000,3767.000000
mean,18.930449,48.192934,7.446265e+03,59.658206,1392.171489,0.832720,15.574359,0.060914,0.486459,0.032877,0.324624,0.093791
std,32.195721,508.212430,2.798259e+05,378.398721,10832.098694,0.304840,28.969821,0.176596,0.134264,0.136346,0.248753,0.228050
min,11.000000,0.000000,0.000000e+00,1.000000,11.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,0.000000,-1.360069
25%,12.000000,4.017241,5.900000e+01,1.000000,15.000000,0.885061,10.900000,-0.028110,0.410640,0.000000,0.000000,-0.022234
50%,14.000000,12.200000,1.920000e+02,1.666667,29.000000,0.987429,12.920000,0.062018,0.487027,0.000000,0.425768,0.098270
75%,19.000000,28.496503,4.900000e+02,8.545455,148.500000,1.000000,17.000000,0.157520,0.565292,0.082398,0.511634,0.217379
max,1103.000000,20629.751004,1.374015e+07,9204.916667,368913.000000,1.000000,1103.000000,1.000000,1.000000,1.000000,1.000000,1.371719


In [67]:
users_DD_top = users_DD[users_DD['score_mean'] > 60]
users_DD_top.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 34 to 7244
Data columns (total 29 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   author              328 non-null    object 
 1   id                  328 non-null    int64  
 2   title               328 non-null    object 
 3   selftext            328 non-null    object 
 4   date                328 non-null    object 
 5   day                 328 non-null    object 
 6   month               328 non-null    object 
 7   year                328 non-null    object 
 8   tickers_            141 non-null    object 
 9   tickers_count_      328 non-null    object 
 10  link_flair_text     328 non-null    object 
 11  comments_mean       328 non-null    float64
 12  comments_sum        328 non-null    int64  
 13  score_mean          328 non-null    float64
 14  score_sum           328 non-null    int64  
 15  upvote_ratio_mean   328 non-null    float64
 16  upvote

In [68]:
users_DD.to_csv("/content/drive/MyDrive/WSB/WSB_users_DD.csv",sep=';')

## Can we rank the users according to their expertise?

In [69]:
## rank users based on upvote ratio, num_comments etc.
## order by score_mean 
users_DD = users_DD.sort_values(by=['score_mean'],ascending=False)
users_DD.head()

,author,id,title,selftext,date,day,month,year,tickers_,tickers_count_,link_flair_text,comments_mean,comments_sum,score_mean,score_sum,upvote_ratio_mean,upvote_ratio_sum,flairs,flairs_count,title_polarity,title_subjectivity,body_polarity,body_subjectivity,title_tokens,selftext_tokens,polarity,years_count,months_count,dates_count
5295,hugh_dickinson,12,MY GME CALLS UP 50K!! HELP Bullish pattern in ...,I'm retarded and I don't know how to sell!!!!!...,"[2021-03-11, 2021-02-19, 2021-02-11, 2021-02-1...",11 19 11 11 2 2 31 30 27 26 15 6,"[3, 2, 2, 2, 2, 2, 1, 1, 12, 12, 12, 9]","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...",NaN,{},YOLO Chart Meme Meme Chart Meme Discussion New...,677.583333,8131,9204.916667,110459,0.990833,11.89,"['YOLO', 'Chart', 'Meme', 'Meme', 'Chart', 'Me...","{'YOLO': 1, 'Chart': 2, 'Meme': 3, 'Discussion...",-0.145238,0.409524,0.043557,0.470348,"['MY', 'GME', 'CALLS', 'UP', '50', 'K', '!', '...","['I', ""'m"", 'retarded', 'and', 'I', 'do', ""n't...",-0.101681,"{'2021': 8, '2020': 4}","{'3': 1, '2': 5, '1': 2, '12': 3, '9': 1}","{'2021-03-11': 1, '2021-02-19': 1, '2021-02-11..."
3967,WalyWal,11,Porn addict hasn’t masturbated in 54 days and ...,[https://i.gyazo.com/1e8bd2b486e77f0b1963a80...,"[2021-02-02, 2021-01-28, 2020-12-25, 2020-12-0...",2 28 25 3 3 3 3 3 3 30 15,"[2, 1, 12, 12, 12, 12, 12, 12, 12, 11, 9]","[2021, 2021, 2020, 2020, 2020, 2020, 2020, 202...","['PLTR', 'GME']","{'PLTR': 1, 'GME': 1}",Loss YOLO Storytime Meme Meme Meme Shitpost Me...,435.727273,4793,7784.727273,85632,0.993636,10.93,"['Loss', 'YOLO', 'Storytime', 'Meme', 'Meme', ...","{'Loss': 1, 'YOLO': 2, 'Storytime': 1, 'Meme':...",0.362500,0.650000,0.055556,0.237778,"['Porn', 'addict', 'has', 'n’t', 'masturbated'...","[' ', '[', 'https://i.gyazo.com/1e8bd2b486e77...",0.418056,"{'2021': 2, '2020': 9}","{'2': 1, '1': 1, '12': 7, '11': 1, '9': 1}","{'2021-02-02': 1, '2021-01-28': 1, '2020-12-25..."
2616,Ordinary-Fox9986,12,GME standoff (exactly how it went down) starri...,**They try to dilute your attention**\n\...,"[2021-02-03, 2021-02-01, 2021-01-31, 2021-01-3...",3 1 31 31 30 29 28 28 25 25 25 18,"[2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...",NaN,{},Meme Meme Meme Discussion Meme Meme Meme Meme ...,493.333333,5920,6296.083333,75553,0.981667,11.78,"['Meme', 'Meme', 'Meme', 'Discussion', 'Meme',...","{'Meme': 10, 'Discussion': 1, 'News': 1}",-0.059722,0.394841,0.112644,0.557268,"['GME', 'standoff', '(', 'exactly', 'how', 'it...","[' ', '*', '*', 'They', 'try', 'to', 'dil...",0.052922,{'2021': 12},"{'2': 2, '1': 10}","{'2021-02-03': 1, '2021-02-01': 1, '2021-01-31..."
6858,thisisshe14,11,$26.77 at close they were trying to keep it un...,As we sit watching robinhood hearing what’...,"[2021-05-27, 2021-02-25, 2021-02-18, 2021-01-3...",27 25 18 31 29 29 28 28 28 27 25,"[5, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...",NaN,{},Discussion Discussion Discussion Chart News Ne...,1078.090909,11859,5862.636364,64489,0.987273,10.86,"['Discussion', 'Discussion', 'Discussion', 'Ch...","{'Discussion': 5, 'Chart': 1, 'News': 4, 'Gain...",0.180392,0.537745,-0.065476,0.172619,"['$', '26.77', 'at', 'close', 'they', 'were', ...","[' ', 'As', 'we', 'sit', 'watching', 'robin...",0.114916,{'2021': 11},"{'5': 1, '2': 2, '1': 8}","{'2021-05-27': 1, '2021-02-25': 1, '2021-02-18..."
5457,jamsi,11,Reddit Celebrated WallStreetBets in a Five-Sec...,,"[2021-02-08, 2021-02-03, 2021-02-02, 2021-02-0...",8 3 2 1 1 1 1 31 30 29 28,"[2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1]","[2021, 2021, 2021, 2021, 2021, 2021, 2021, 202...","['BB', 'AMC', 'GME', 'NOK']","{'BB': 1, 'AMC': 1, 'GME': 1, 'NOK': 1}",News News News News News News News News Discus...,510.090909,5611,5769.000000,63459,0.971818,10.69,"['News', 'News', 'News', 'News', 'News', 'News...","{'News': 10, 'Discussion': 1}",0.055761,0.454252,0.000000,0.000000,"['Reddit', 'Celebrated', 'WallStreetBets', 'in...",[' '],0.055761,{'2021': 11},"{'2': 7, 

## Are the same users active in the other reviewed subreddits?

In [ ]:
## compare users with investing and stocks